In [1]:
from konlpy.tag import Twitter
from sklearn.metrics import mean_absolute_error
import numpy as np
import pickle

twitter = Twitter()

In [3]:
with open("data/data_ml.pkl", "rb") as f:
    index2voca = pickle.load(f)
    voca2index = pickle.load(f)
    train_X = pickle.load(f)
    train_y = pickle.load(f)
    test_X = pickle.load(f)
    test_y = pickle.load(f)

vocaNum = len(index2voca)
trainNum = len(train_y)
testNum = len(test_y)

In [5]:
class dict_based():
    def __init__(self, voca2index):
        self.voca2index = voca2index
        self.vocaNum = len(self.voca2index)
        self.sentiment_dict = None
        
    def fit(self, train_X, train_y):
        self.sentiment_dict = np.zeros(self.vocaNum)
        word_freq = np.zeros(self.vocaNum)
        for i,j in zip(*train_X.nonzero()):
            self.sentiment_dict[j] += train_y[i]
            word_freq[j] += 1
        
        self.sentiment_dict = np.divide(self.sentiment_dict, word_freq)
            
    def predict(self, test_input):
        if type(test_input) == str:
            test_vec = np.zeros((1, self.vocaNum))
            for i, w in enumerate(twitter.morphs(test_input)):
                test_vec[0, voca2index.get(w, voca2index['<UNK>'])] += 1

            return np.divide(test_vec.dot(self.sentiment_dict), test_vec.sum(axis=1).T)[0]
        else:
            return np.divide(test_input.dot(self.sentiment_dict), test_input.sum(axis=1).T).T

In [6]:
model = dict_based(voca2index)
model.fit(train_X, train_y)

In [7]:
predicted = model.predict(test_X)
print("mean absolute error: {}".format(mean_absolute_error(test_y, predicted)))

mean absolute error: 0.24832614987235874


In [8]:
test_seq = "영화 재미없다"
predicted = model.predict(test_seq)
print("predict: {}".format(predicted))

predict: 0.41497780130372824


In [ ]:
with open("dict_based.pkl", 'wb') as f:
    pickle.dump(model, f)